# V. SAI SHRUTHIK (PRASUNET-ML-3)

In [2]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm

import joblib
from sklearn.model_selection import GridSearchCV
import cv2
import seaborn as sns
import time
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [3]:
confusion_image_path = 'confusion matrix.png'
classification_file_path = 'classification_report.txt'
model_file_path = "svm_model.pkl"
train_dir = "C:/Users/natsu/OneDrive/Desktop/PRASUNET/TASK3/train"
test_dir = "C:/Users/natsu/OneDrive/Desktop/PRASUNET/TASK3/ttest1"
train_images = os.listdir(train_dir)
features = []
labels = []
image_size = (50, 50)

In [4]:
for image in tqdm(train_images, desc="Processing Train Images"):
    if image[0:3] == 'cat' :
        label = 0
    else :
        label = 1
    image_read = cv2.imread(train_dir+"/"+image)
    image_resized = cv2.resize(image_read, image_size)
    image_normalized = image_resized / 255.0
    image_flatten = image_normalized.flatten()
    features.append(image_flatten)
    labels.append(label)

Processing Train Images: 100%|██████████████████████████████████████████████████| 19017/19017 [03:09<00:00, 100.59it/s]


In [5]:
del train_images

In [6]:
features = np.asarray(features)
labels = np.asarray(labels)

# train test split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True, random_state=42)

In [7]:
del features
del labels

In [8]:
# PCA, SVM, & Pipeline
n_components = 0.8
pca = PCA(n_components=n_components)
svm = SVC()
pca = PCA(n_components=n_components, random_state=42)
pipeline = Pipeline([
    ('pca', pca),
    ('svm', svm)
])

In [9]:
param_grid = {
    'pca__n_components': [2, 1, 0.9, 0.8],
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
}

In [10]:
# training
start_time = time.time()

grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=4)
grid_search.fit(X_train, y_train)


end_time = time.time()

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END pca__n_components=2, svm__kernel=linear;, score=0.657 total time=20.1min
[CV 2/3] END pca__n_components=2, svm__kernel=linear;, score=0.657 total time=10.4min
[CV 3/3] END pca__n_components=2, svm__kernel=linear;, score=0.657 total time=15.0min
[CV 1/3] END pca__n_components=2, svm__kernel=rbf;, score=0.657 total time=  14.9s
[CV 2/3] END pca__n_components=2, svm__kernel=rbf;, score=0.657 total time=  12.9s
[CV 3/3] END pca__n_components=2, svm__kernel=rbf;, score=0.657 total time=   8.4s
[CV 1/3] END pca__n_components=2, svm__kernel=poly;, score=0.657 total time=   7.3s
[CV 2/3] END pca__n_components=2, svm__kernel=poly;, score=0.657 total time=   7.3s
[CV 3/3] END pca__n_components=2, svm__kernel=poly;, score=0.657 total time=   6.4s
[CV 1/3] END pca__n_components=2, svm__kernel=sigmoid;, score=0.540 total time=   7.2s
[CV 2/3] END pca__n_components=2, svm__kernel=sigmoid;, score=0.545 total time=   6.6s
[CV 3/

In [11]:
del X_train
del y_train

In [12]:
# Calculate total training time
training_time = end_time - start_time

# Convert time to hours, minutes, seconds
hours, remainder = divmod(training_time, 3600)
minutes, seconds = divmod(remainder, 60)

# Print out the training time
print(f"Training time: {int(hours)} hour {int(minutes)} minute {seconds:.2f} second")

Training time: 5 hour 33 minute 34.83 second


In [13]:
# Get the best parameters & models
best_pipeline = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best Score: ", best_score)

Best Parameters:  {'pca__n_components': 0.9, 'svm__kernel': 'rbf'}
Best Score:  0.7099191480970223


In [14]:
# Save the best model
model_filename = 'best_model.joblib'
joblib.dump(best_pipeline, model_filename)

print(f"Model saved as {model_filename}")

Model saved as best_model.joblib


In [15]:
# Evaluation with test data set
accuracy = best_pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7252891692954785


In [ ]:
y_pred = best_pipeline.predict(X_test)

# classification report
target_names = ['Cat', 'Dog']
classification_rep = classification_report(y_test, y_pred, target_names=target_names)
print("Classification Report:\n", classification_rep)

with open(classification_file_path, 'w') as file:
    file.write(classification_rep)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.savefig(confusion_image_path)
plt.show()